In [78]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [79]:
# Libraries to use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.cluster import KMeans
import talib as tb
from scipy import stats

In [80]:
%matplotlib inline
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [81]:
path = 'C:/Users/LZJF_02/Desktop/BacktestRA/'

In [82]:
filenameKKTest = 'KkRatioStrategy20170601-20171201.csv'
filenameKKTrain = 'KkRatioStrategy20160601-20170601.csv'

In [83]:
# multi
filenameMultiCycleTest = 'MultiCycleStrategy20170601-20171201.csv'
filenameMultiCycleTrain =  'MultiCycleStrategy20150601-20170601.csv' 

In [84]:
def calPosPeriod(df,barBin = 5):
    df.entryDt = df.entryDt.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df.exitDt = df.exitDt.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df['posPeriod'] = (df.exitDt - df.entryDt)
    df['posPeriod'] = df['posPeriod'].apply(lambda x: x.total_seconds() / 60 /barBin)
    return df

In [85]:
def varDiff(df1,df2,iterNum = 10000):
    """ To test the whether there is a significance difference bewteen two population variance with unequal Sample size.
    Parameters:
    df1:DataFrame. The outcome of calPosPeriod function.
    df2:DataFrame. The outcome of calPosPeriod function.
    iterNum: Int. The iteration number.
    Return:Series. The Series of pValueList.
    """
    shape1 = df1.shape[0]
    shape2 = df2.shape[0]
    pValueList = []
    if shape1 > shape2:
        for i in xrange(iterNum):
            pValueList.append(stats.ttest_rel(np.random.choice(df1.pnl.values,shape2),
                                              df2.pnl.values).pvalue)
    else:
        for i in xrange(iterNum):
            pValueList.append(stats.ttest_rel(np.random.choice(df2.pnl.values,shape1),
                                              df1.pnl.values).pvalue)
    return pd.Series(pValueList)

In [86]:
kkTest = pd.read_csv(path+filenameKKTest,infer_datetime_format=True,parse_dates=[0])
kkTrain = pd.read_csv(path+filenameKKTrain,infer_datetime_format=True,parse_dates=[0])
kkTest = calPosPeriod(kkTest)
kkTrain = calPosPeriod(kkTrain)

In [87]:
MultiCycleTest = pd.read_csv(path+filenameMultiCycleTest,infer_datetime_format=True,parse_dates=[0])
MultiCycleTrain = pd.read_csv(path+filenameMultiCycleTrain,infer_datetime_format=True,parse_dates=[0])
MultiCycleTest = calPosPeriod(MultiCycleTest)
MultiCycleTrain = calPosPeriod(MultiCycleTrain)

In [88]:
P_MultiCycle = varDiff(MultiCycleTest,MultiCycleTrain)

In [105]:
print 'The 5% quantile is'+" "+ str(P_MultiCycle.quantile(0.05)),'\t'
print 'The number of P-Values below 0.05 is' +" "+ str(P_MultiCycle[P_MultiCycle <= 0.05].shape[0] / P_MultiCycle.shape[0])
P_MultiCycle.describe()

The 5% quantile is 0.235650109537 	
The number of P-Values below 0.05 is 0.0007


count    10000.000000
mean         0.652901
std          0.229176
min          0.010044
25%          0.487959
50%          0.681834
75%          0.845903
max          0.999945
dtype: float64

In [141]:
MultiCycleTest.describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,66.000000,66.000000,66.000000,66.000000,66.0,66.000000,66.000000
mean,3706.893939,0.090909,73995.151515,22.198545,20.0,57.498424,127.142424
std,233.609215,1.003490,4600.183689,1.380055,0.0,483.688124,182.816585
min,2962.000000,-1.000000,59470.000000,17.841000,20.0,-474.411000,2.400000
25%,3610.000000,-1.000000,72040.000000,21.612000,20.0,-278.011750,20.600000
50%,3728.000000,1.000000,74365.000000,22.309500,20.0,-37.783500,78.500000
75%,3860.750000,1.000000,77030.000000,23.109000,20.0,186.013250,118.050000
max,4084.000000,1.000000,81410.000000,24.423000,20.0,2386.393000,860.000000


In [150]:
MultiCycleTest.pnl.std()**2 / MultiCycleTrain.pnl.std()**2

1.6233477447214497

In [173]:
# The Levene test tests the null hypothesis that all input samples are from populations with equal variances. 
# Levene’s test is an alternative to Bartlett’s test bartlett in the case where there are significant deviations from normality.
stats.levene(MultiCycleTest.pnl.values, MultiCycleTrain.pnl.values)

LeveneResult(statistic=1.6006632447902718, pvalue=0.20678134160909584)

In [157]:
# F检定判断两者样本方差是否一样（Note that the F-test is extremely sensitive to non-normality of X and Y, so you're probably better off doing a more robust 
#test such as Levene's test or Bartlett's test unless you're reasonably sure that X and Y are distributed normally.）
stats.f.cdf(MultiCycleTest.pnl.std()**2 / MultiCycleTrain.pnl.std()**2, MultiCycleTest.shape[0]-1,MultiCycleTrain.shape[0]-1)

0.99518945616117949

In [134]:
# 多头利润统计分析
MultiCycleTestLong = MultiCycleTest[MultiCycleTest.volume ==1]
MultiCycleTestLong.describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,36.000000,36.0,36.000000,36.000000,36.0,36.000000,36.000000
mean,3717.777778,1.0,74133.333333,22.240000,20.0,179.982222,176.933333
std,232.130273,0.0,4555.145913,1.366544,0.0,579.812098,231.717449
min,3150.000000,1.0,63070.000000,18.921000,20.0,-474.411000,4.600000
25%,3621.250000,1.0,72455.000000,21.736500,20.0,-178.389500,44.750000
50%,3742.500000,1.0,74595.000000,22.378500,20.0,47.014000,88.100000
75%,3869.500000,1.0,77355.000000,23.206500,20.0,520.518250,239.600000
max,4056.000000,1.0,81370.000000,24.411000,20.0,2386.393000,860.000000


In [138]:
MultiCycleTestLong[MultiCycleTestLong.pnl > 0].describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,20.000000,20.0,20.000000,20.000000,20.0,20.000000,20.000000
mean,3770.150000,1.0,74848.000000,22.454400,20.0,512.545600,245.600000
std,196.088668,0.0,3873.626887,1.162088,0.0,583.714444,253.773243
min,3262.000000,1.0,64010.000000,19.203000,20.0,6.315000,4.600000
25%,3665.000000,1.0,72762.500000,21.828750,20.0,108.019000,84.600000
50%,3757.500000,1.0,74595.000000,22.378500,20.0,358.268000,102.400000
75%,3951.750000,1.0,78755.000000,23.626500,20.0,657.617000,284.950000
max,4056.000000,1.0,79720.000000,23.916000,20.0,2386.393000,860.000000


In [140]:
MultiCycleTestLong[MultiCycleTestLong.pnl < 0].describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,16.000000,16.0,16.000000,16.000000,16.0,16.00000,16.000000
mean,3652.312500,1.0,73240.000000,21.972000,20.0,-235.72200,91.100000
std,262.292894,0.0,5279.829543,1.583949,0.0,145.07835,171.941432
min,3150.000000,1.0,63070.000000,18.921000,20.0,-474.41100,4.600000
25%,3520.250000,1.0,70695.000000,21.208500,20.0,-372.03350,17.750000
50%,3689.500000,1.0,74065.000000,22.219500,20.0,-207.83150,48.300000
75%,3806.000000,1.0,76427.500000,22.928250,20.0,-139.66350,78.600000
max,4047.000000,1.0,81370.000000,24.411000,20.0,-30.15100,719.400000


In [123]:
# 空头利润统计分析
MultiCycleTestShort= MultiCycleTest[MultiCycleTest.volume ==-1]
MultiCycleTestShort.describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,30.000000,30.0,30.000000,30.000000,30.0,30.000000,30.000000
mean,3693.833333,-1.0,73829.333333,22.148800,20.0,-89.482133,67.393333
std,238.669022,0.0,4726.193287,1.417858,0.0,278.695858,57.781431
min,2962.000000,-1.0,59470.000000,17.841000,20.0,-433.409000,2.400000
25%,3606.250000,-1.0,71920.000000,21.576000,20.0,-341.516250,15.850000
50%,3718.000000,-1.0,74230.000000,22.269000,20.0,-106.793500,65.500000
75%,3858.500000,-1.0,76935.000000,23.080500,20.0,100.752250,105.900000
max,4084.000000,-1.0,81410.000000,24.423000,20.0,538.022000,234.400000


In [127]:
MultiCycleTestShort[MultiCycleTestShort.pnl>0].describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,9.000000,9.0,9.000000,9.000000,9.0,9.000000,9.000000
mean,3627.888889,-1.0,72856.666667,21.857000,20.0,257.031889,119.933333
std,281.985569,0.0,5661.384548,1.698415,0.0,184.269655,49.216054
min,2962.000000,-1.0,59470.000000,17.841000,20.0,17.494000,53.000000
25%,3574.000000,-1.0,71650.000000,21.495000,20.0,137.440000,96.600000
50%,3634.000000,-1.0,73260.000000,21.978000,20.0,192.159000,112.200000
75%,3751.000000,-1.0,75200.000000,22.560000,20.0,406.393000,127.600000
max,3912.000000,-1.0,78690.000000,23.607000,20.0,538.022000,234.400000


In [128]:
MultiCycleTestShort[MultiCycleTestShort.pnl<0].describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,21.000000,21.0,21.000000,21.000000,21.0,21.000000,21.000000
mean,3722.095238,-1.0,74246.190476,22.273857,20.0,-237.988143,44.876190
std,219.075992,0.0,4354.285792,1.306286,0.0,147.825094,45.800021
min,3035.000000,-1.0,60390.000000,18.117000,20.0,-433.409000,2.400000
25%,3610.000000,-1.0,72010.000000,21.603000,20.0,-363.076000,8.000000
50%,3720.000000,-1.0,74250.000000,22.275000,20.0,-302.344000,18.800000
75%,3862.000000,-1.0,76940.000000,23.082000,20.0,-101.948000,79.000000
max,4084.000000,-1.0,81410.000000,24.423000,20.0,-1.270000,155.600000


In [93]:
pValueList = []
for i in xrange(10000):
    pValueList.append(stats.ttest_rel(np.random.choice(kkTrain.pnl.values,min(kkTest.shape[0],kkTrain.shape[0])), kkTest.pnl.values).pvalue)   
pValueSeries = pd.Series(pValueList)
pValueSeries.describe()

count    10000.000000
mean         0.569243
std          0.260137
min          0.003496
25%          0.359017
50%          0.582893
75%          0.790181
max          0.999990
dtype: float64

In [94]:
PValue = varDiff(kkTrain,kkTest)

In [95]:
PValue.describe()

count    10000.000000
mean         0.568433
std          0.261260
min          0.009622
25%          0.355666
50%          0.581790
75%          0.791658
max          0.999952
dtype: float64

In [96]:
pValueSeries[pValueSeries <= 0.05].shape[0] / pValueSeries.shape[0]

0.0068

In [97]:
pValueSeries.quantile(0.05)

0.13459499876408176

In [98]:
# 根据pValue的均值和中位数以及5%的分位数可以看出 P-Value是大概率远大于0.05的，所以接受原假设，及两组PNL方差相同，收益稳定。